In [1]:
import itertools


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [3]:
import glob
import random
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset
from PIL import Image


In [4]:
#!pip install visdom


In [5]:
import visdom
import numpy as np
# vis = visdom.Visdom()
# vis.text('Hello, world!')
# vis.image(np.ones((3, 10, 10)))

In [6]:


from data import *
from models import *
from utils import *
import math


In [7]:
#TRAIN OPTIONS FROM GATED GAN
epoch = 0
n_epochs = 200 #default = 200
decay_epoch=100
batchSize = 1
dataroot = './photo2fourcollection'
loadSize = 143
fineSize = 128
ngf = 64
ndf = 64    
in_nc = 3 
out_nc = 3 
# niter = 100  
# niter_decay = 100 
lr = 0.0002 
beta1 = 0.5 
#ntrain = math.huge 
flip = 1  
display_id = 10 
display_winsize = 128 
display_freq = 25 
gpu = 1 
# name = ''   
which_direction = 'AtoB'
phase = 'train'
lambda_A = 10.0
use_lsgan = True
align_data = 0
pool_size = 50
resize_or_crop = 'resize_and_crop'
autoencoder_constrain = 10 
n_styles = 4
test_data_path = ''

cuda=False
tv_strength=1e-6


In [34]:
dataloader = DataLoader(ImageDataset('./photo2fourcollection'), 
                        batch_size=1, shuffle=True, num_workers=4)
batch = next(iter(dataloader))


In [35]:
batch['style_label']

tensor([2])

In [36]:
generator = Generator(in_nc, out_nc, n_styles, ngf)
discriminator= Discriminator(in_nc, n_styles, ndf)

In [37]:
#generator.load_state_dict(torch.load('./output/netG.pth'))

In [38]:
if cuda:
    generator.cuda()
    discriminator.cuda()

In [39]:
#Losses Init
use_lsgan=True
if use_lsgan:
    criterion_GAN = nn.MSELoss()
else: 
    criterion_GAN = nn.BCELoss()
    
    
criterion_ACGAN = nn.CrossEntropyLoss()
criterion_Rec = nn.L1Loss()
#criterion_Enc = nn.MSELoss()
criterion_TV = TVLoss(TVLoss_weight=tv_strength)




In [40]:
#Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(generator.parameters(),
                                lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), 
                               lr=lr, betas=(0.5, 0.999))


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch,decay_epoch).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda=LambdaLR(n_epochs,epoch, decay_epoch).step)



In [41]:
#Set vars for training
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
input_A = Tensor(batchSize, in_nc, fineSize, fineSize)
input_B = Tensor(batchSize, out_nc, fineSize, fineSize)
target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)

# print(Tensor(batch['style']))
D_A_size = discriminator(input_A.copy_(batch['style']))[0].size()  
D_AC_size = discriminator(input_B.copy_(batch['style']))[1].size()
# print(D_AC_size)

class_label_B = Tensor(D_AC_size[0],D_AC_size[1],D_AC_size[2]).long()
# print(class_label_B.shape)

autoflag_OHE = Tensor(1,n_styles+1).fill_(0).long()
autoflag_OHE[0][-1] = 1

fake_label = Tensor(D_A_size).fill_(0.0)
real_label = Tensor(D_A_size).fill_(0.99) 

rec_A_AE = Tensor(batchSize,in_nc,fineSize,fineSize)

fake_buffer = ReplayBuffer()

##INIT THOSE WEIGHTS INIT
generator.apply(weights_init_normal)


Generator(
  (encoder): Encoder(
    (model): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace)
      (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace)
    )
  )
  (transformer): Transformer(
    (t): ModuleList(
      (0): ResidualBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_runnin

In [42]:
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace)
  )
  (fldiscriminator): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (aux_clf): Conv2d(512, 4, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
)

In [43]:
logger = Logger(n_epochs, len(dataloader))


In [44]:
# #source image
# real_content = Variable(input_A.copy_(batch['content']))
# #target style
# real_style = Variable(input_B.copy_(batch['style']))
# #simple label of style index
# style_label = batch['style_label']
# #one hot encoded style label
# style_OHE = F.one_hot(style_label,n_styles).long()
# #
# class_label = class_label_B.copy_(label2tensor(style_label,class_label_B)).long()

# #
# #rec_AE = T

In [45]:
### TRAIN LOOP
for epoch in range(epoch,n_epochs):
    for i, batch in enumerate(dataloader):
        real_content = Variable(input_A.copy_(batch['content']))
        real_style = Variable(input_B.copy_(batch['style']))
        style_label = batch['style_label']
        style_OHE = F.one_hot(style_label,n_styles).long()
        
        class_label = class_label_B.copy_(label2tensor(style_label,class_label_B)).long()
        
        #### Update Discriminator
        optimizer_D.zero_grad()
        
        #err_Rec.backward()

        #Fake Loss
        genfake = generator({
            'content':real_content,
            'style_label': style_OHE,
        })
        
        fake = fake_buffer.push_and_pop(genfake)
        out_gan, out_class = discriminator(fake)
        errD_fake = criterion_GAN(out_gan, fake_label)
        #Discriminator Classifier is not trained on fake images
        errD_fake.backward()
        
        optimizer_D.step()
        
        #Real Loss
        optimizer_D.zero_grad()
        out_gan, out_class = discriminator(real_style)

        errD_real_class = criterion_ACGAN(out_class.transpose(1,3),class_label)*lambda_A
        #errD_real_class.backward(retain_graph=True)

        errD_real = criterion_GAN(out_gan, real_label)
        #errD_real.backward()
        
        errD_real_total = errD_real + errD_real_class
        errD_real_total.backward()
        
        optimizer_D.step()
        
        errD = (errD_real+errD_fake)/2.0
        
                
        #### Generator Update
        optimizer_G.zero_grad()
        out_gan, out_class = discriminator(genfake)
        
        err_gan = criterion_GAN(out_gan, real_label)
        err_class = criterion_ACGAN(out_class.transpose(1,3), class_label)*lambda_A
        
        err_TV = criterion_TV(genfake)
        
        errG_tot = err_gan + err_class + err_TV 
        
        errG_tot.backward()
        optimizer_G.step()
        
        optimizer_G.zero_grad()
        identity = generator({
            'content': real_content,
            'style_label': autoflag_OHE,
        })
        err_ae = criterion_Rec(identity,real_content)*autoencoder_constrain
        err_ae.backward()
        optimizer_G.step()
        
        
        #Progress report (port 8097)
        logger.log({'Generator Loss': errG_tot,
                    'Reconstruction Loss': err_ae,
                    'loss_G_GAN': err_gan,
                    'loss_G_AC': err_class,
                    'Discriminator GAN Loss': errD,
                    'tv_loss':err_TV,
                    'Discriminator Class Loss': errD_real_class}, 
                    images={'content': real_content,
                            'style': real_style,
                            'transfer': genfake,
                            'auto-reconstruction':identity})
    
    ##update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D.step()
    
    #Save model
    torch.save(generator.state_dict(), 'output/netG.pth')
    torch.save(discriminator.state_dict(), 'output/netD.pth')

        


Epoch 001/200 [0008/6287] -- Generator Loss: 15.4983 | Reconstruction Loss: 5.8463 | loss_G_GAN: 1.2120 | loss_G_AC: 14.2479 | Discriminator GAN Loss: 2.0556 | tv_loss: 0.0384 | Discriminator Class Loss: 20.4351 -- ETA: 27 days, 6:56:21.7396846

KeyboardInterrupt: 